## .envファイルの作成

次のセルを実行して、環境変数を格納するための`.env`ファイルを作成してください。セルの実行は、セルの左上の再生マークをクリックすることでできます。

In [ ]:
%%bash

cd ..

# .envファイルを作成する
cat <<EOF > .env2
SEARCH_SERVICE_ENDPOINT="your_search_service_endpoint"
SEARCH_QUERY_KEY="your_search_query_key"
SEARCH_API_KEY="your_search_api_key"
AOAI_ENDPOINT="your_aoai_endpoint"
AOAI_API_VERSION=2023-10-01-preview
AOAI_API_KEY="your_aoai_api_key"
EOF

echo ".envファイルが作成されました。"

## 環境変数の設定
Azure Portalで各環境変数を確認し、先ほど作成した`.env`ファイルに格納してください。

1. `SEARCH_SERVICE_ENDPOINT,SEARCH_QUERY_KEY,SEARCH_API_KEY`の設定
    - `SEARCH_SERVICE_ENDPOINT`は、AI Searchリソースの「概要」タブから確認できます。
    ![Image 1](../assets/env1.png)

    - `SEARCH_QUERY_KEY,SEARCH_API_KEY`は、AI Searchリソースの「設定」>「キー」タブから確認できます。
    ![Image 2](../assets/env2.png)

2. `AOAI_ENDPOINT,AOAI_API_KEY`の設定
    - `AOAI_ENDPOINT,AOAI_API_KEY`は、OpenAI Servicesリソースの「リソース管理」>「キーとエンドポイント」タブから確認できます。
    ![Image 3](../assets/env3.png)

#### **※ここまで完了されたら、以降順番にセルを実行してRAGの実装を体験してみてください。**

## 必要なライブラリのインポート
以下のセルで必要なライブラリをインポートします

In [1]:
import os
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import *
from langchain.text_splitter import RecursiveCharacterTextSplitter
from azure.search.documents.models import VectorizedQuery
from PyPDF2 import PdfReader
from openai import AzureOpenAI

## 環境変数の読み込み
envファイルから環境変数を読み込みます。

In [5]:
# 環境変数からAzure AI Search、Azure OpenAIのエンドポイント等を取得する
load_dotenv()
search_endpoint = os.environ["SEARCH_SERVICE_ENDPOINT"]
search_api_key = os.environ["SEARCH_API_KEY"]
search_query_key = os.environ["SEARCH_QUERY_KEY"]
aoai_endpoint = os.environ["AOAI_ENDPOINT"]
aoai_api_version = os.environ["AOAI_API_VERSION"]
aoai_api_key = os.environ["AOAI_API_KEY"]


## Azure AI Searchのインデックスを作成する

1. **クライアントの生成**: `SearchIndexClient`を使用して、Azure AI Searchのインデックスクライアントを作成します。
2. **インデックスの確認**: インデックスがすでに存在する場合、再作成を避けるために何もしません。
3. **フィールドの定義**: インデックスに含まれるフィールドを定義します。ここでは、ドキュメントID、コンテンツ、コンテンツベクトルのフィールドを設定します。
4. **ベクトル検索の設定**: ベクトル検索の設定を行います。
5. **インデックスの作成**: 定義した設定を用いてインデックスを作成します。

以下のコードを実行して、インデックスを作成します。



In [6]:
def create_index():
    """
    Azure AI Searchのインデックスを作成する
    """
    client = SearchIndexClient(endpoint= search_endpoint, credential=AzureKeyCredential(search_api_key))
    name = "docs"

    # すでにインデックスが作成済みである場合には何もしない
    if 'docs' in client.list_index_names():
        print("すでにインデックスが作成済みです")
        return

    # インデックスのフィールドを定義する
    fields = [
        SimpleField(name="id", type=SearchFieldDataType.String, key=True),
        SearchableField(name="content", type="Edm.String", analyzer_name="ja.microsoft"),
        SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                searchable=True, vector_search_dimensions=1536, vector_search_profile_name="myHnswProfile")
    ]

    # ベクトル検索のための定義を行う
    vector_search = VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw"
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
            )
        ]
    )

    # インデックスを作成する
    index = SearchIndex(name=name, fields=fields, vector_search=vector_search)
    client.create_index(index)

# インデックスを作成する
create_index()


## ドキュメントをAzure AI Searchにインデクシングする

1. **PDFからテキストを抽出**: `PdfReader`を使用してPDFファイルからテキストを抽出します。ここでは、ドキュメントととして厚生労働省が提供する[モデル就業規則](https://www.mhlw.go.jp/content/001018385.pdf)を使用します。
2. **テキストのチャンク化**: テキストを指定したサイズでチャンクに分割します。これにより、大きなテキストを小さな部分に分けて処理しやすくします。
3. **インデクシング**: チャンク化されたテキストをAzure AI Searchにインデックスします。ここでは、Azure OpenAIを使用してテキストのベクトルを生成し、それを含むドキュメントをAzure AI Searchにアップロードします。

以下のコードを順次実行して、ドキュメントをインデックスします。


In [ ]:
def extract_text_from_docs(filepath):
    """
    PDFからテキストを抽出する
    """
    print(f"{filepath}内のテキストを抽出中...")
    with open(filepath, "rb") as f:
        reader = PdfReader(f)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text

# 対象ファイルパスのファイルを読み込んで、Azure AI Searchにインデックスする
filepath = "/workspaces/aoai-rag-handson/data/001018385.pdf"  # 対象ファイルパス
content = extract_text_from_docs(filepath)

In [8]:
def create_chunk(content: str, separator: str, chunk_size: int = 512, overlap: int = 0):
    """
    テキストを指定したサイズで分割する
    """
    splitter = RecursiveCharacterTextSplitter(chunk_overlap=overlap, chunk_size=chunk_size, separators=separator)
    chunks = splitter.split_text(content)
    return chunks

# テキストを指定したサイズで分割する
chunksize = 1000  # チャンクサイズ
overlap = 200  # オーバーラップサイズ
separator = ["\n\n", "\n", "。", "、", " ", ""]  # 区切り文字
chunks = create_chunk(content, separator, chunksize, overlap)
print("チャンク化完了")

In [ ]:
def index_docs(chunks: list):
    """
    ドキュメントをAzure AI Searchにインデックスする
    """
    # Azure AI SearchのAPIに接続するためのクライアントを生成する
    searchClient = SearchClient(
        endpoint=search_endpoint,
        index_name="docs",
        credential=AzureKeyCredential(search_api_key)
    )

    # Azure OpenAIのAPIに接続するためのクライアントを生成する
    openAIClient = AzureOpenAI(
        api_key=aoai_api_key,
        api_version=aoai_api_version,
        azure_endpoint=aoai_endpoint
    )

    # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
    for i, chunk in enumerate(chunks):
        print(f"{i+1}個目のチャンクを処理中...")
        response = openAIClient.embeddings.create(
            input=chunk,
            model="text-embedding-3-small-deploy"
        )

        # チャンク化されたテキストとそのテキストのベクトルをAzure AI Searchにアップロードする
        document = {"id": str(i), "content": chunk, "contentVector": response.data[0].embedding}
        searchClient.upload_documents([document])

# テキストをAzure AI Searchにインデックスする
index_docs(chunks)

## プロンプトのベクトル化

1. **OpenAIクライアントの作成**: OpenAI APIに接続するためのクライアントを作成します。
2. **プロンプトのベクトル化**: 指定したプロンプトをOpenAIモデルを使用してベクトル化します。このベクトルは、後の検索クエリとして使用されます。`prompt`については、任意のものに変更していただいても構いません。

以下のコードを実行して、プロンプトをベクトル化します。


In [ ]:
# OpenAIクライアントの作成
openAIClient = AzureOpenAI(
    api_key=aoai_api_key,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint
)

# プロンプトをベクトル化する関数
def generate_embeddings(prompt, model="text-embedding-3-small-deploy"): # model = "deployment_name"
    response = openAIClient.embeddings.create(input=prompt, model=model).data[0].embedding
    return response

# プロンプトをベクトル化
prompt = "就業時間に関してどのような規定があるのか重要度順に3つ教えてください"
vectorized_prompt = generate_embeddings(prompt)
print(vectorized_prompt[:50])

## ベクトル検索

1. **SearchClientの作成**: Azure AI Searchに接続するためのクライアントを作成します。
2. **ベクトル検索の準備**: ベクトルクエリを作成し、指定されたフィールドで上位の結果を取得します。
3. **検索実行と結果取得**: 検索を実行し、最初の検索結果を取得します。

以下のコードを実行して、ベクトル検索を行います。


In [ ]:
# Azure AI SearchのAPIに接続するためのクライアントを生成する
searchClient = SearchClient(
    endpoint=search_endpoint,
    index_name="docs",
    credential=AzureKeyCredential(search_api_key)
)

# ベクトルクエリの作成
vector_query = VectorizedQuery(
    vector=vectorized_prompt,
    k_nearest_neighbors=3,  # 上位3件の結果を取得します
    fields="contentVector"  # ベクトル検索を行うフィールドを指定します
)

# ベクトル検索の実行
results = searchClient.search(
    search_text='',  # ベクトル検索のみ行うためテキストクエリは空
    vector_queries=[vector_query],
    select=['id', 'content'],
)

# 最初の検索結果を取得
first_result = next(results, None)
if first_result:
    print(first_result["content"])
else:
    print("検索結果が見つかりませんでした")


## Azure OpenAIに回答生成依頼

1. **システムメッセージの定義**: GPT-3.5に対するシステムメッセージを定義し、AIのキャラクターや回答スタイルを設定します。
2. **ユーザーメッセージの作成**: 検索クエリと検索結果を含むユーザーメッセージを作成します。
3. **回答生成の依頼**: Azure OpenAIに対して、ユーザーメッセージに基づいた回答を生成するよう依頼します。
4. **回答の表示**: 生成された回答を表示します。

以下のコードを実行して、ベクトル検索の結果に基づいた回答を生成します。


In [ ]:
# Azure OpenAIクライアントの作成
openAIClient = AzureOpenAI(
    api_key=aoai_api_key,
    api_version=aoai_api_version,
    azure_endpoint=aoai_endpoint
)

# システムメッセージの定義
system_message_chat_conversation = """
あなたはユーザーの質問に回答するチャットボットです。
回答については、「Sources:」以下に記載されている内容に基づいて回答してください。
回答は簡潔にしてください。
「Sources:」に記載されている情報以外の回答はしないでください。
また、ユーザーの質問に対して、Sources:以下に記載されている内容に基づいて適切な回答ができない場合は、「すみません。わかりません。」と回答してください。
回答の中に情報源の提示は含めないでください。例えば、回答の中に「Sources:」という形で情報源を示すことはしないでください。
"""

# ユーザーメッセージの作成
user_message = """
{query}

Sources:
{source}
""".format(query=prompt, source=first_result["content"])

# メッセージリストの作成
messages_for_vector_answer = [
    {"role": "system", "content": system_message_chat_conversation},
    {"role": "user", "content": user_message}
]

# Azure OpenAI Serviceに回答生成を依頼
response = openAIClient.chat.completions.create(
    model="gpt-35-turbo-deploy",
    messages=messages_for_vector_answer
)

# 生成された回答を表示
response_text = response.choices[0].message.content
print(response_text)
